In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import os
from torch_setupParameters import set_seed,set_device, getDefaultRNNArgs
from torch_dataPreprocessing import loadAllRealDatasets, prepareDataCubesForRNN
from torch_dataPreprocessing import normalizeSentenceDataCube, binTensor
from torch_dataPreprocessing import handBCI_Dataset, handBCI_SythDataset, gaussSmooth
from tfrecord.torch.dataset import MultiTFRecordDataset


from tqdm.notebook import tqdm
from pathlib import Path
from time import time

from tfrecord.torch.dataset import TFRecordDataset

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [7]:
SEED = 2021
set_seed(seed=SEED)
DEVICE = set_device()

Random seed 2021 has been set.
GPU is enabled !


In [10]:
args['outputDir']

'output/RNNTrainingSteps/HeldOutTrials'

In [13]:
#point this towards the top level dataset directory
rootDir = '../handwritingBCIData/'
outDir = 'output/'
#train an RNN using data from these specified sessions
dataDirs = ['t5.2019.05.08','t5.2019.11.25','t5.2019.12.09','t5.2019.12.11','t5.2019.12.18',
            't5.2019.12.20','t5.2020.01.06','t5.2020.01.08','t5.2020.01.13','t5.2020.01.15']


#use this train/test partition 
cvPart = 'HeldOutTrials'

#name of the directory where this RNN run will be saved
rnnOutputDir = cvPart

## parameters
args = getDefaultRNNArgs(rootDir, cvPart, outDir)
#Configure the arguments for a multi-day RNN (that will have a unique input layer for each day)
for x in range(len(dataDirs)):
    args['sentencesFile_'+str(x)] = '/media/rich/bigSSD/NMA_data/Willet_data/Datasets/'+dataDirs[x]+'/sentences.mat'
    args['singleLettersFile_'+str(x)] = '/media/rich/bigSSD/NMA_data/Willet_data/Datasets/'+dataDirs[x]+'/singleLetters.mat'
    args['labelsFile_'+str(x)] = '/media/rich/bigSSD/NMA_data/Willet_data/RNNTrainingSteps/Step2_HMMLabels/'+cvPart+'/'+dataDirs[x]+'_timeSeriesLabels.mat'
    args['syntheticDatasetDir_'+str(x)] = '/media/rich/bigSSD/NMA_data/Willet_data/Datasets/'+dataDirs[x]+'/'+cvPart+'/'+dataDirs[x]+'_syntheticSentences/'
    args['cvPartitionFile_'+str(x)] = '/media/rich/bigSSD/NMA_data/Willet_data/RNNTrainingSteps/trainTestPartitions_'+cvPart+'.mat'
    args['sessionName_'+str(x)] = dataDirs[x]
args['outputDir'] = '/media/rich/bigSSD/NMA_data/Willet_data/RNN_outputs'

for t in range(30):  ## 10 days
    if 'labelsFile_'+str(t) not in args.keys():
        args['nDays'] = t
        break
if not os.path.isdir(args['outputDir']):
    os.mkdir(args['outputDir'])
    
#this weights each day equally (0.1 probability for each day) and allocates a unique input layer for each day (0-9)
args['dayProbability'] = '[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]'
args['dayToLayerMap'] = '[0,1,2,3,4,5,6,7,8,9]'
# args['verbose'] = True ## extra print-out information

args['mode'] = 'train' ## make sure it is set in 'train' mode
print('batchSize:', args['batchSize'])
print('synthBatchSize:', args['synthBatchSize'])
args['ForTestingOnly'] = False ## FOR DEBUGING. set "self.nDays = 2" (use 2 days of data for testing run)


batchSize: 8
synthBatchSize: 3


## torch dataloaders

In [ ]:
#define which datasets to process
dataDirs = ['t5.2019.05.08',
            't5.2019.11.25',
            't5.2019.12.09',
            't5.2019.12.11',
            't5.2019.12.18',
            't5.2019.12.20',
            't5.2020.01.06',
            't5.2020.01.08',
            't5.2020.01.13',
#             't5.2020.01.15'
           ]
cvParts = ['HeldOutBlocks', 'HeldOutTrials']

n_epochs = 4
n_days = len(dataDirs)
n_bats = 20
batch_size = 256
val_fraction = 0.2

win_len = 500

n_electrodes = 192
n_letters = 32

dir_folders = Path(r'/media/rich/bigSSD/NMA_data/Willet_data/Datasets/').resolve()

DEVICE = set_device()
net = Net().to(DEVICE)
for ii, layer in enumerate(net.parameters()):
    if type(layer) == nn.Linear:
        nn.init.xavier_uniform_(layer, gain=nn.init.calculate_gain('relu'))
criterion_letters = nn.CrossEntropyLoss()
criterion_onsets = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), 
                             lr=0.0002, 
#                              betas=(0.9, 0.999), 
#                              eps=1e-08, 
                             weight_decay=0.0001, 
                             amsgrad=False
                            )

In [14]:
allSynthDataLoaders = []
allRealDataLoaders = []
allValDataLoaders = []
daysWithValData = []
args['isTraining'] = True
for dayIdx in [2,3]: #range(args['nDays']):
    ## real data
    print('Loading real data ', dayIdx)
    neuralData, targets, errWeights, binsPerTrial, cvIdx = prepareDataCubesForRNN(args['sentencesFile_'+str(dayIdx)],
                                                                          args['singleLettersFile_'+str(dayIdx)],
                                                                          args['labelsFile_'+str(dayIdx)],
                                                                          args['cvPartitionFile_'+str(dayIdx)],
                                                                          args['sessionName_'+str(dayIdx)],
                                                                          args['rnnBinSize'],
                                                                          args['timeSteps'],
                                                                          args['isTraining'])
    realDataSize = args['batchSize'] - args['synthBatchSize']
    trainIdx = cvIdx['trainIdx']
    valIdx = cvIdx['testIdx']
    print('create real dataset ', dayIdx)
    realData_train = handBCI_Dataset(args,neuralData[trainIdx,:,:], targets[trainIdx,:,:], errWeights[trainIdx,:],\
                               binsPerTrial[trainIdx,np.newaxis],\
                               addNoise=True)
    realDataTrain_Loader = torch.utils.data.DataLoader(realData_train, batch_size =realDataSize,shuffle=True, num_workers=0)
    
    if len(valIdx)==0:
        realDataVal_Loader = realDataTrain_Loader
    else:
        realData_val = handBCI_Dataset(args,neuralData[valIdx,:,:], targets[valIdx,:,:], errWeights[valIdx,:],\
                                   binsPerTrial[valIdx,np.newaxis],\
                                       addNoise=False)
        realDataVal_Loader = torch.utils.data.DataLoader(realData_val, batch_size =args['batchSize'],shuffle=True, num_workers=0)
        daysWithValData.append(dayIdx)
    allRealDataLoaders.append(realDataTrain_Loader)
    allValDataLoaders.append(realDataVal_Loader)
              
    ## sythetic data
    print('processing sythetic data ', dayIdx)
    synthDir = args['syntheticDatasetDir_'+str(dayIdx)]
    synth_obj = handBCI_SythDataset(synthDir, args)
    synth_ds = synth_obj.makeDataSet()
    synth_loader = torch.utils.data.DataLoader(synth_ds, batch_size=args['synthBatchSize'])
    allSynthDataLoaders.append(synth_loader)
  

Loading real data  2
create real dataset  2
processing sythetic data  2
Loading real data  3
create real dataset  3
processing sythetic data  3


In [37]:
class charSeqNet(nn.Module):
    def __init__(self, args):
        super(charSeqNet, self).__init__()
        """
        """        
        #count how many days of data are specified
        self.nDays = args['nDays']
        self.args = args
        if self.args['seed']==-1:
            self.args['seed']=datetime.now().microsecond
        drop_prob = args['drop_prob']
        #define the dimensions of layers in the RNN
        nOutputs = 31
        nInputs = 192        
        nUnits = args['nUnits']
        nTimeSteps = args['timeSteps']
        self.rnnBinSize = args['rnnBinSize']
        inputLayers = []        
#        shape: [args['batchSize'], args['timeSteps'], nInputs]
        for j in range(self.nDays):
            inputLayers.append(nn.Linear(nInputs, nInputs, bias = True))

        self.inputLayers = inputLayers
        self.gru1 = torch.nn.GRU(nInputs, nUnits, 1, \
                                 batch_first=True, dropout=drop_prob)
 
        self.gru2 = torch.nn.GRU(nUnits, nUnits, 1, \
                                 batch_first=True, dropout=drop_prob)

        self.fc1 = nn.Linear(nUnits, nOutputs, bias = True)
        
        self.criterion = nn.CrossEntropyLoss()
 
    def forward(self, x, dayIdx):

        if self.args['smoothInputs']==1: ## smooth
            x = torch.Tensor(gaussSmooth(x, kernelSD=4/self.rnnBinSize))
        layer1 = self.inputLayers[dayIdx].to('cuda') ## day specific input layer
        x = layer1(x.to('cuda'))
        x, h = self.gru1(x)
        ## TODO: downsample x's time dimension
        x, h = self.gru2(x)
        ## TODO: upsample
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)
    
    def train(self):
        ## TO be continued ...
        n = 100
        for epoc in range(n):
            dayIdx = np.random.randint(self.nDays)
            miniBatch = next(iter(allRealDataLoaders[0])) ## change 0 to dayIdx in full mode
            X = miniBatch['inputs']
            target = miniBatch['labels'].to(DEVICE).long()
            er = miniBatch['errWeights']
            output = self.forward(X, dayIdx)
            ## output is 32 dimensions, target is 31?
            loss = self.criterion(output, target[:,-1,:31])
            loss.backward()

In [38]:
# miniBatch = next(iter(allRealDataLoaders[0]))

In [39]:
## Uncomment the lines below to train your network
charSeq_net = charSeqNet(args).to(DEVICE)
print("Total Parameters in Network {:10d}".format(sum(p.numel() for p in charSeq_net.parameters())))

Total Parameters in Network    2676255


/media/rich/OS/Users/Richard/Linux_stuff_on_OS/conda_envs/envs/rapids-0.19/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [40]:
charSeq_net.train()

/media/rich/OS/Users/Richard/Linux_stuff_on_OS/conda_envs/envs/rapids-0.19/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/media/rich/Home_Linux_partition/github_repos/NMA-project/MH_handBCI_pytorch/torch_dataPreprocessing.py:208: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data[:,:,np.newaxis]).transpose(1,2)
